In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kausthubkannan/5-flower-types-classification-dataset

 94% 227M/242M [00:02<00:00, 101MB/s]
100% 242M/242M [00:02<00:00, 108MB/s]


In [5]:
import zipfile
import os

# Gantilah nama file zip dengan nama yang sesuai jika diperlukan
zip_path = '5-flower-types-classification-dataset.zip'
extract_path = 'dataset'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path ke direktori dataset utama
dataset_path = 'dataset/flower_images/'

# Path ke direktori baru untuk menyimpan subset train dan validation
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'validation')

# Jika direktori train dan validation belum ada, buat direktorinya
for directory in [train_dir, validation_dir]:
    os.makedirs(directory, exist_ok=True)

# List folder flower
flower_folder = ['Lilly', 'Lotus', 'Orchid', 'Sunflower', 'Tulip']

# Fungsi untuk memindahkan gambar ke direktori baru
def move_images(images, destination_dir):
    for image_path in images:
        image_name = os.path.basename(image_path)
        category_name = os.path.basename(os.path.dirname(image_path))
        target_dir = os.path.join(destination_dir, category_name)
        os.makedirs(target_dir, exist_ok=True)
        shutil.move(image_path, os.path.join(target_dir, image_name))

# Memindahkan gambar ke direktori train dan validation untuk setiap flower
for flo_folder in flower_folder:
    # Path ke direktori flower
    flower_path = os.path.join(dataset_path, flo_folder)

    # List semua gambar di direktori flower
    all_images = [os.path.join(flower_path, image) for image in os.listdir(flower_path)]

    # Membagi dataset untuk setiap flower
    train_images, validation_images = train_test_split(all_images, test_size=0.2, random_state=42)

    # Memindahkan gambar ke direktori train
    move_images(train_images, train_dir)

    # Memindahkan gambar ke direktori validation
    move_images(validation_images, validation_dir)

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Ubah nilai ini sesuai dengan ukuran gambar yang ada dalam dataset Anda
img_height, img_width = 150, 150
# Ubah nilai ini sesuai dengan jumlah kelas (kategori Beras)
num_classes = 5
# Ubah nilai ini sesuai dengan preferensi Anda
batch_size = 32
# Ubah nilai ini sesuai dengan jumlah epoch yang diinginkan
num_epochs = 20

# Path ke direktori dataset
dataset_path = 'dataset/flower_images/'

# Langkah 2: Persiapkan Data untuk Model
train_dir = os.path.join(dataset_path, 'train')
validation_dir = os.path.join(dataset_path, 'validation')

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 4000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [8]:
# Langkah 3: Bangun dan Latih Model CNN
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

Epoch 1/20
125/125 [==============================] - 210s 2s/step - loss: 1.2980 - accuracy: 0.4650 - val_loss: 1.0413 - val_accuracy: 0.6180
Epoch 2/20
125/125 [==============================] - 208s 2s/step - loss: 1.0356 - accuracy: 0.5888 - val_loss: 1.0062 - val_accuracy: 0.6030
Epoch 3/20
125/125 [==============================] - 204s 2s/step - loss: 0.9177 - accuracy: 0.6453 - val_loss: 0.8549 - val_accuracy: 0.6760
Epoch 4/20
125/125 [==============================] - 199s 2s/step - loss: 0.8074 - accuracy: 0.6870 - val_loss: 0.8607 - val_accuracy: 0.6740
Epoch 5/20
125/125 [==============================] - 197s 2s/step - loss: 0.7218 - accuracy: 0.7247 - val_loss: 0.7660 - val_accuracy: 0.7260
Epoch 6/20
125/125 [==============================] - 215s 2s/step - loss: 0.6152 - accuracy: 0.7695 - val_loss: 0.7662 - val_accuracy: 0.7310
Epoch 7/20
125/125 [==============================] - 196s 2s/step - loss: 0.5624 - accuracy: 0.7922 - val_loss: 0.7223 - val_accuracy: 0.7540

In [11]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path ke gambar baru yang ingin diuji
new_image_path = 'orchid2.jpg'

# Muat dan pra-pemrosesan gambar
img = image.load_img(new_image_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalisasi

# Gunakan model untuk memprediksi kelas
predictions = model.predict(img_array)

# Ambil nilai probabilitas untuk setiap kelas
probabilities = predictions[0]

# Print nilai probabilitas untuk setiap kelas
for i, prob in enumerate(probabilities):
    print(f'Probability for class {i}: {prob:.4f}')

# Dapatkan indeks kelas dengan probabilitas tertinggi
predicted_class = np.argmax(probabilities)

# Dapatkan label kelas dari generator gambar
class_labels = {v: k for k, v in train_generator.class_indices.items()}

# Dapatkan label kelas yang diprediksi
predicted_class_label = class_labels[predicted_class]

# Print hasil prediksi sebagai angka (indeks kelas)
print(f'Predicted class (numeric): {predicted_class}')

# Print hasil prediksi sebagai nama kelas
print(f'Predicted class (label): {predicted_class_label}')

1/1 [==============================] - 0s 31ms/step
Probability for class 0: 0.0000
Probability for class 1: 0.0000
Probability for class 2: 1.0000
Probability for class 3: 0.0000
Probability for class 4: 0.0000
Predicted class (numeric): 2
Predicted class (label): Orchid


In [12]:
# Menyimpan model dengan nama "model" dalam format HDF5
model.save('flower_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
